## Importing Required Libraries

In [12]:
import pandas as pd
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from skimage import io, color, img_as_ubyte
import os
import cv2

In [13]:
df = pd.DataFrame(
    columns=[
        "Contrast Feature",
        "Dissimilarity Feature",
        "Homogeneity Feature",
        "Energy Feature",
        "Correlation Feature",
        "ASM Feature",
    ]
)

In [14]:
matrix1 = []

### Matrix

In [15]:
path_of_images = r"./images"
list_of_images = os.listdir(path_of_images)
for image in list_of_images:
    img = cv2.imread(os.path.join(path_of_images, image))
    gray = color.rgb2gray(img)
    image = img_as_ubyte(gray)  # Becomes 8-bit unsigned integer

    # This step is similar to data compression, because the 8-bit image contains 256 gray levels, which will cause the calculation of the gray level co-occurrence matrix to be too large, so it is compressed into 16 levels and the gray levels are divided
    bins = np.array(
        [0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240, 255]
    )  # 16-bit
    inds = np.digitize(
        image, bins
    )  # Returns a matrix with the same size as the image, but the matrix element represents the interval position of the element in the image in the bins, less than 0 is 0, 0-16 is 1, and so on

    max_value = inds.max() + 1
    matrix_coocurrence = graycomatrix(
        inds,  # Numpy matrix for co-occurrence matrix calculation
        [1],  # Step
        [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4],  # Direction angle
        levels=max_value,  # Co-occurrence matrix order
        normed=False,
        symmetric=False,
    )
    # P[i,j,d,theta] returns a four-dimensional matrix, each dimension represents a different meaning
    matrix1.append(matrix_coocurrence)

In [16]:
CF =[]
DF =[]
HF =[]
EF =[]
COR = []
ASM = []

# GLCM properties


In [17]:
for matrix in matrix1:

   def contrast_feature(matrix):
      contrast = graycoprops(matrix)
      return list(contrast)

   def dissimilarity_feature(matrix):
      dissimilarity = graycoprops(matrix)
      return list(dissimilarity)

   def homogeneity_feature(matrix):
      homogeneity = graycoprops(matrix)
      return list(homogeneity)

   def energy_feature(matrix):
      energy = graycoprops(matrix)
      return list(energy)

   def correlation_feature(matrix):
      correlation = graycoprops(matrix)
      return list(correlation)

   def asm_feature(matrix):
      asm = graycoprops(matrix)
      return list(asm)

   CF.append(np.mean(contrast_feature(matrix)))
   DF.append(np.mean(dissimilarity_feature(matrix)))
   HF.append(np.mean(homogeneity_feature(matrix)))
   EF.append(np.mean(energy_feature(matrix)))
   COR.append(np.mean(correlation_feature(matrix)))
   ASM.append(np.mean(asm_feature(matrix)))

In [18]:
Features = [CF, DF, HF, EF, COR, ASM]
for i, j in zip(df.columns, Features):
    df[i] = j

In [19]:
df.index = ["Image1", "Image2", "Image3"]
df.index.name = "Image ID"

In [20]:
df

,Contrast Feature,Dissimilarity Feature,Homogeneity Feature,Energy Feature,Correlation Feature,ASM Feature
Image ID,,,,,,
Image1,0.748796,0.748796,0.748796,0.748796,0.748796,0.748796
Image2,0.675464,0.675464,0.675464,0.675464,0.675464,0.675464
Image3,0.563571,0.563571,0.563571,0.563571,0.563571,0.563571


In [22]:
df.to_csv("Feature Extraction.csv")